In [2]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load dataset
data = sm.datasets.get_rdataset('dietox', 'geepack').data

data.head()

,Pig,Evit,Cu,Litter,Start,Weight,Feed,Time
0,4601,Evit000,Cu000,1,26.5,26.50000,NaN,1
1,4601,Evit000,Cu000,1,26.5,27.59999,5.200005,2
2,4601,Evit000,Cu000,1,26.5,36.50000,17.600000,3
3,4601,Evit000,Cu000,1,26.5,40.29999,28.500000,4
4,4601,Evit000,Cu000,1,26.5,49.09998,45.200001,5


In [3]:
'''
Right now we save a mock dataset as a .csv file for the analysis jupyter notebook. Normally, within this notebook
we would connect to the smartshark instance, run our queries that generate the data, wrangle this data into a 
pandas dataframe, and then save it as a csv. 

However, this step is left as a excercise.
'''

data.to_csv("pred_data.csv", index=False)

In [10]:
from pymongo import MongoClient
import pandas as pd

connection = MongoClient(
    host="research.cassee.dev",
    username="read-shark",
    password="msr2021shark")

db = connection.smartshark_2_1

projects = list(db.project.find())
pr_systems = list(db.pull_request_system.find())

'''
We connect to the database and run the following query. This querys selects
the merged pull requests and when they were opened and closed
'''
pull_requests = list(db.pull_request.find({'merged_at': {'$exists': True }}, {'created_at': 1, 'merged_at' : 1, 'pull_request_system_id': 1, 'author_association': 1}))

print(f"Found {len(pull_requests)} pull requests")

Found 16779 pull requests


In [13]:
def count_comments_for_a_pr(pr):
    '''
    Find the number of comments for a PR by counting all elements in the pull_request_comment collection
    that have an entry for which the pull_request_id field is set to the identifier of the pr
    '''
    pr["comment_count"] = db.pull_request_comment.count_documents({'pull_request_id': pr["_id"]})

In [11]:
'''
We count the number of comments per PR for each PR. As this fires off 
one query per PR this might take a minute or two. If this takes too 
long, you can always consider stopping this loop after thousand iterations,
and first testing out your entire pipeline before continuing.
'''

for pr in pull_requests:
    if "comment_count" not in pr:
        count_comments_for_a_pr(pr)

In [12]:
'''
Create two dictionaries that can be used to traverse from a PR system id to a project name in constant time
'''
vcs_to_project_id = {pr_sys["_id"]:pr_sys["project_id"] for pr_sys in pr_systems}
project_id_to_name = {proj["_id"] : proj["name"] for proj in projects}

'''
Convert our PR data to a matrix where each row is a PR. We only convert
the PRs for which comment_count is set.
'''
rows = [[pr["created_at"].weekday() > 4, 
         (pr["merged_at"] - pr["created_at"]).total_seconds(), 
         project_id_to_name[vcs_to_project_id[pr["pull_request_system_id"]]], 
         pr["author_association"],
         pr["comment_count"]] 
                for pr in pull_requests if "comment_count" in pr]

df = pd.DataFrame(rows, columns = ["IsWeekday", "Duration", "Project", "AuthorAssociation", "CommentCount"])

df.head()

print(f"DF has {len(df.index)} rows")

df.to_csv('se.csv', index=False)

DF has 16779 rows
